# 🐦 Klasyfikacja Gatunków Ptaków — Transfer Learning

**Autor:** Wygenerowano z pomocą Claude (Anthropic)  
**Framework:** PyTorch + torchvision  
**Model bazowy:** EfficientNet-B0 (pretrenowany na ImageNet)

## Spis treści
1. Wprowadzenie — czym jest Transfer Learning?
2. Instalacja i import bibliotek
3. Przygotowanie danych
4. Augmentacja i transformacje
5. Budowa modelu z Transfer Learningiem
6. Faza 1: Feature Extraction
7. Faza 2: Fine-Tuning
8. Ewaluacja modelu
9. Predykcja na nowych obrazach
10. Podsumowanie

## 1. Wprowadzenie — Czym jest Transfer Learning?

### Problem
Trening od zera wymaga ogromnych zbiorów danych, dużej mocy obliczeniowej i czasu.

### Rozwiązanie
Transfer learning przenosi wiedzę z modelu wytrenowanego na dużym zbiorze (ImageNet — 1.2M obrazów, 1000 klas) do nowego zadania.

### Analogia
Ucząc się rozpoznawać ptaki, nie zaczynasz od zera — już wiesz jak wyglądają kształty, kolory, tekstury. Transfer learning działa tak samo!

### Dwie fazy
| Faza | Opis | Learning Rate |
|------|-------|---------------|
| **Feature Extraction** | Zamrożone warstwy bazowe, trenujemy nowy klasyfikator | 0.001 |
| **Fine-Tuning** | Odmrożone warstwy, delikatne dostrajanie | 0.0001 |

## 2. Instalacja i import bibliotek

In [ ]:
# Instalacja (odkomentuj jeśli potrzebujesz)
# !pip install torch torchvision matplotlib pillow numpy

In [ ]:
import os, copy, time, json
from pathlib import Path
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Urządzenie: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 3. Przygotowanie danych

Dane powinny mieć strukturę:
```
data/
├── train/  (70%)
│   ├── American_Robin/
│   ├── Blue_Jay/
│   └── ...
├── val/    (15%)
└── test/   (15%)
```

**Polecane datasety:**
- [Birds 525 Species (Kaggle)](https://www.kaggle.com/datasets/gpiosenka/100-bird-species)
- [CUB-200-2011](https://www.vision.caltech.edu/datasets/cub_200_2011/)

In [ ]:
# === KONFIGURACJA ===
DATA_DIR = './data'
OUTPUT_DIR = './output'
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 20
FREEZE_EPOCHS = 5
LEARNING_RATE = 0.001
FINE_TUNE_LR = 0.0001
PATIENCE = 5
NUM_WORKERS = 2

os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Konfiguracja gotowa!')

In [ ]:
# === DANE DEMO (usuń gdy masz prawdziwe dane!) ===
import random

BIRD_SPECIES = [
    'American_Robin', 'Blue_Jay', 'Cardinal', 'Chickadee', 'Crow',
    'Eagle', 'Falcon', 'Goldfinch', 'Hawk', 'Heron',
    'Hummingbird', 'Kingfisher', 'Magpie', 'Nightingale', 'Oriole',
    'Owl', 'Parrot', 'Pelican', 'Penguin', 'Robin',
    'Sparrow', 'Starling', 'Swan', 'Woodpecker', 'Wren'
]

def create_demo_data(data_dir, species_list, images_per_class=20):
    splits = {'train': int(images_per_class * 0.7),
              'val': max(2, int(images_per_class * 0.15)),
              'test': max(2, int(images_per_class * 0.15))}
    for split, count in splits.items():
        for species in species_list:
            d = os.path.join(data_dir, split, species)
            os.makedirs(d, exist_ok=True)
            br, bg, bb = hash(species)%200+30, hash(species+'g')%200+30, hash(species+'b')%200+30
            for i in range(count):
                arr = np.random.normal(loc=[br,bg,bb], scale=[30,30,30], size=(256,256,3)).clip(0,255).astype(np.uint8)
                Image.fromarray(arr).save(os.path.join(d, f'{species}_{i:04d}.jpg'))
        print(f'  {split}: {count} x {len(species_list)} = {count*len(species_list)} obrazów')

if not os.path.exists(os.path.join(DATA_DIR, 'train')):
    print('Generowanie danych demo...')
    create_demo_data(DATA_DIR, BIRD_SPECIES)
    print('Gotowe! (Zastąp prawdziwymi zdjęciami)')
else:
    print('Dane istnieją!')

## 4. Augmentacja i transformacje

**Augmentacja** sztucznie zwiększa różnorodność danych treningowych. Stosujemy ją TYLKO na danych treningowych! Normalizacja ImageNet jest kluczowa przy transfer learningu.

In [ ]:
# === TRANSFORMACJE ===
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'val': transforms.Compose([
        transforms.Resize(IMAGE_SIZE + 32), transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(), transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'test': transforms.Compose([
        transforms.Resize(IMAGE_SIZE + 32), transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(), transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
}
print('Transformacje gotowe!')

In [ ]:
# === ŁADOWANIE DANYCH ===
image_datasets, dataloaders, dataset_sizes = {}, {}, {}
for split in ['train', 'val', 'test']:
    d = os.path.join(DATA_DIR, split)
    if os.path.exists(d):
        image_datasets[split] = datasets.ImageFolder(d, transform=data_transforms[split])
        dataloaders[split] = DataLoader(image_datasets[split], batch_size=BATCH_SIZE,
                                         shuffle=(split=='train'), num_workers=NUM_WORKERS, pin_memory=True)
        dataset_sizes[split] = len(image_datasets[split])
        print(f'{split}: {dataset_sizes[split]} obrazów')

class_names = image_datasets['train'].classes
num_classes = len(class_names)
print(f'\nGatunki ({num_classes}): {class_names[:5]}...')

In [ ]:
# === WIZUALIZACJA PRÓBKI ===
inputs, labels = next(iter(dataloaders['train']))
fig, axes = plt.subplots(2, 4, figsize=(16, 7))
for i, ax in enumerate(axes.flatten()):
    if i >= len(inputs): break
    img = inputs[i].numpy().transpose((1,2,0))
    img = np.array(IMAGENET_STD) * img + np.array(IMAGENET_MEAN)
    ax.imshow(np.clip(img, 0, 1)); ax.set_title(class_names[labels[i]], fontsize=10); ax.axis('off')
plt.suptitle('Próbka danych treningowych', fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()

## 5. Budowa modelu

### Dlaczego EfficientNet-B0?
| Model | Parametry | ImageNet Top-1 |
|-------|-----------|----------------|
| ResNet-50 | 25.6M | 76.1% |
| **EfficientNet-B0** | **5.3M** | **77.1%** |
| VGG-16 | 138M | 71.6% |

Najlepszy stosunek jakości do rozmiaru!

### Architektura
```
EfficientNet-B0 (zamrożony) → Dropout → Linear(1280→512) → ReLU → Dropout → Linear(512→25)
```

In [ ]:
# === BUDOWA MODELU ===
weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
model = models.efficientnet_b0(weights=weights)
print('Załadowano wagi ImageNet')

# Zamrażamy warstwy bazowe
for param in model.parameters():
    param.requires_grad = False

# Nowa warstwa klasyfikacyjna
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3), nn.Linear(num_features, 512),
    nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(512, num_classes))

model = model.to(device)
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Parametry: {total:,} łącznie, {trainable:,} trenowalnych ({trainable/total*100:.1f}%)')

## 6. Faza 1: Feature Extraction

Warstwy bazowe **zamrożone**. Trenujemy TYLKO nową warstwę klasyfikacyjną. Model uczy się mapować cechy z EfficientNet na nasze gatunki.

In [ ]:
# === FUNKCJE TRENINGOWE ===
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience, self.min_delta = patience, min_delta
        self.counter, self.best_score, self.should_stop = 0, None, False
    def __call__(self, val_score):
        if self.best_score is None: self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience: self.should_stop = True; print('  Early Stopping!')
        else: self.best_score = val_score; self.counter = 0

def train_epoch(model, dl, criterion, opt, device, size):
    model.train(); loss_sum, correct = 0.0, 0
    for x, y in dl:
        x, y = x.to(device), y.to(device)
        opt.zero_grad(); out = model(x); loss = criterion(out, y)
        _, pred = torch.max(out, 1); loss.backward(); opt.step()
        loss_sum += loss.item()*x.size(0); correct += torch.sum(pred == y.data)
    return loss_sum/size, (correct.double()/size).item()

def val_epoch(model, dl, criterion, device, size):
    model.eval(); loss_sum, correct = 0.0, 0
    with torch.no_grad():
        for x, y in dl:
            x, y = x.to(device), y.to(device)
            out = model(x); loss = criterion(out, y); _, pred = torch.max(out, 1)
            loss_sum += loss.item()*x.size(0); correct += torch.sum(pred == y.data)
    return loss_sum/size, (correct.double()/size).item()

print('Funkcje treningowe gotowe!')

In [ ]:
# === FAZA 1 ===
print('='*50); print('FAZA 1: Feature Extraction'); print('='*50)

criterion = nn.CrossEntropyLoss()
opt1 = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
sch1 = optim.lr_scheduler.ReduceLROnPlateau(opt1, mode='max', factor=0.5, patience=2)
es = EarlyStopping(patience=PATIENCE)
h1 = defaultdict(list); best_acc = 0.0; best_wts = copy.deepcopy(model.state_dict())

for epoch in range(FREEZE_EPOCHS):
    t = time.time()
    tl, ta = train_epoch(model, dataloaders['train'], criterion, opt1, device, dataset_sizes['train'])
    vl, va = val_epoch(model, dataloaders['val'], criterion, device, dataset_sizes['val'])
    print(f'Epoka {epoch+1}/{FREEZE_EPOCHS} [{time.time()-t:.1f}s] Train: {ta:.4f} | Val: {va:.4f}')
    h1['train_loss'].append(tl); h1['train_acc'].append(ta); h1['val_loss'].append(vl); h1['val_acc'].append(va)
    sch1.step(va)
    if va > best_acc: best_acc = va; best_wts = copy.deepcopy(model.state_dict()); print(f'  Najlepszy: {best_acc:.4f}')
    es(va)
    if es.should_stop: break

model.load_state_dict(best_wts)
print(f'\nFaza 1 zakończona! Best: {best_acc:.4f}')

## 7. Faza 2: Fine-Tuning

Odmrażamy ostatnie warstwy i trenujemy z **niższym LR**. Warstwy bazowe: 10x niższy LR (delikatna korekta).

In [ ]:
# === FAZA 2 ===
print('='*50); print('FAZA 2: Fine-Tuning'); print('='*50)

for layer in list(model.features.children())[-3:]:
    for param in layer.parameters(): param.requires_grad = True

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'Trenowalnych: {trainable:,} / {total:,} ({trainable/total*100:.1f}%)')

opt2 = optim.Adam([
    {'params': model.features.parameters(), 'lr': FINE_TUNE_LR * 0.1},
    {'params': model.classifier.parameters(), 'lr': FINE_TUNE_LR}])
sch2 = optim.lr_scheduler.ReduceLROnPlateau(opt2, mode='max', factor=0.5, patience=2)
es = EarlyStopping(patience=PATIENCE)
h2 = defaultdict(list); ft_epochs = NUM_EPOCHS - FREEZE_EPOCHS

for epoch in range(ft_epochs):
    t = time.time()
    tl, ta = train_epoch(model, dataloaders['train'], criterion, opt2, device, dataset_sizes['train'])
    vl, va = val_epoch(model, dataloaders['val'], criterion, device, dataset_sizes['val'])
    print(f'Epoka {epoch+1}/{ft_epochs} [{time.time()-t:.1f}s] Train: {ta:.4f} | Val: {va:.4f}')
    h2['train_loss'].append(tl); h2['train_acc'].append(ta); h2['val_loss'].append(vl); h2['val_acc'].append(va)
    sch2.step(va)
    if va > best_acc: best_acc = va; best_wts = copy.deepcopy(model.state_dict()); print(f'  Najlepszy: {best_acc:.4f}')
    es(va)
    if es.should_stop: break

model.load_state_dict(best_wts)
print(f'\nFine-tuning zakończony! Best: {best_acc:.4f}')

In [ ]:
# === WYKRES HISTORII ===
tl = h1['train_loss']+h2['train_loss']; vl = h1['val_loss']+h2['val_loss']
ta = h1['train_acc']+h2['train_acc']; va = h1['val_acc']+h2['val_acc']
eps = range(1, len(tl)+1); p1 = len(h1['train_loss'])

fig, (a1, a2) = plt.subplots(1, 2, figsize=(14, 5))
a1.plot(eps, tl, 'b-', label='Train', lw=2); a1.plot(eps, vl, 'r-', label='Val', lw=2)
if p1<len(eps): a1.axvline(x=p1, color='gray', ls='--', alpha=0.7, label='Fine-tuning')
a1.set_title('Loss'); a1.set_xlabel('Epoka'); a1.legend(); a1.grid(True, alpha=0.3)

a2.plot(eps, ta, 'b-', label='Train', lw=2); a2.plot(eps, va, 'r-', label='Val', lw=2)
if p1<len(eps): a2.axvline(x=p1, color='gray', ls='--', alpha=0.7, label='Fine-tuning')
a2.set_title('Accuracy'); a2.set_xlabel('Epoka'); a2.legend(); a2.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=150); plt.show()

## 8. Ewaluacja modelu

Mierzymy: **Accuracy**, **Precision**, **Recall**, **F1-Score** i tworzymy **macierz pomyłek**.

In [ ]:
# === EWALUACJA ===
model.eval(); all_preds, all_labels = [], []
with torch.no_grad():
    for x, y in dataloaders['test']:
        x = x.to(device); out = model(x); _, pred = torch.max(out, 1)
        all_preds.extend(pred.cpu().numpy()); all_labels.extend(y.numpy())

all_preds, all_labels = np.array(all_preds), np.array(all_labels)
test_acc = np.mean(all_preds == all_labels)
print(f'Test Accuracy: {test_acc:.4f} ({test_acc*100:.1f}%)\n')

print(f'{"Gatunek":<25} {"Prec":>8} {"Recall":>8} {"F1":>8}')
print('-'*51)
for i, name in enumerate(class_names):
    tp = np.sum((all_preds==i) & (all_labels==i))
    fp = np.sum((all_preds==i) & (all_labels!=i))
    fn = np.sum((all_preds!=i) & (all_labels==i))
    p = tp/(tp+fp) if (tp+fp)>0 else 0; r = tp/(tp+fn) if (tp+fn)>0 else 0
    f1 = 2*p*r/(p+r) if (p+r)>0 else 0
    print(f'  {name:<23} {p:>8.3f} {r:>8.3f} {f1:>8.3f}')

In [ ]:
# === CONFUSION MATRIX ===
cm = np.zeros((num_classes, num_classes), dtype=int)
for t, p in zip(all_labels, all_preds): cm[t][p] += 1

fig, ax = plt.subplots(figsize=(max(12, num_classes*0.5), max(10, num_classes*0.4)))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(num_classes), yticks=np.arange(num_classes),
       xticklabels=class_names, yticklabels=class_names,
       title='Macierz pomyłek', ylabel='Prawdziwa', xlabel='Przewidziana')
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
thresh = cm.max()/2
for i in range(num_classes):
    for j in range(num_classes):
        ax.text(j, i, str(cm[i,j]), ha='center', va='center',
                color='white' if cm[i,j]>thresh else 'black', fontsize=7)
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150); plt.show()

## 9. Predykcja na nowych obrazach

Funkcja `predict_bird()` klasyfikuje nowe zdjęcia ptaków i zwraca Top-5 predykcji z pewnością.

In [ ]:
# === PREDYKCJA ===
def predict_bird(image_path, model, class_names, transform, device):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    inp = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(inp); probs = torch.nn.functional.softmax(out, dim=1)
        top5p, top5i = torch.topk(probs, min(5, len(class_names)))
    
    print(f'\nPredykcja dla: {image_path}'); print('-'*40)
    for i in range(top5p.size(1)):
        n = class_names[top5i[0][i].item()]; c = top5p[0][i].item()
        print(f'  {n:<20} {c:.1%} {chr(9608)*int(c*30)}')
    
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(10, 4))
    a1.imshow(img); a1.set_title(f'Predykcja: {class_names[top5i[0][0].item()]}'); a1.axis('off')
    names = [class_names[top5i[0][i].item()] for i in range(top5p.size(1))]
    confs = [top5p[0][i].item() for i in range(top5p.size(1))]
    a2.barh(names[::-1], confs[::-1], color='steelblue'); a2.set_xlim(0,1); a2.set_title('Top 5')
    plt.tight_layout(); plt.show()

# Przykład (odkomentuj):
# predict_bird('ścieżka/do/zdjęcia.jpg', model, class_names, data_transforms['test'], device)
print('Funkcja predict_bird() gotowa!')

## 10. Zapis modelu i podsumowanie

In [ ]:
# === ZAPIS ===
model_path = os.path.join(OUTPUT_DIR, 'bird_classifier.pth')
torch.save({'model_state_dict': model.state_dict(), 'class_names': class_names,
            'num_classes': num_classes, 'test_accuracy': test_acc}, model_path)
print(f'Model zapisany: {model_path}')

with open(os.path.join(OUTPUT_DIR, 'metadata.json'), 'w') as f:
    json.dump({'class_names': class_names, 'num_classes': num_classes,
               'test_accuracy': float(test_acc)}, f, indent=2)
print('Metadane zapisane!')

In [ ]:
# === ŁADOWANIE ZAPISANEGO MODELU ===
def load_saved_model(path, device):
    ckpt = torch.load(path, map_location=device)
    m = models.efficientnet_b0(weights=None)
    nf = m.classifier[1].in_features
    m.classifier = nn.Sequential(
        nn.Dropout(0.3), nn.Linear(nf, 512), nn.ReLU(),
        nn.Dropout(0.2), nn.Linear(512, ckpt['num_classes']))
    m.load_state_dict(ckpt['model_state_dict'])
    m = m.to(device); m.eval()
    return m, ckpt['class_names']

# model, classes = load_saved_model('output/bird_classifier.pth', device)
print('Gotowe!')

## Podsumowanie

### Co zrobiliśmy:
1. Załadowaliśmy pretrenowany EfficientNet-B0
2. Feature Extraction — trening nowej warstwy klasyfikacyjnej
3. Fine-Tuning — delikatne dostrojenie całego modelu
4. Ewaluacja na zbiorze testowym
5. Funkcja predykcji dla nowych obrazów

### Dalsze kroki:
- **Prawdziwe dane**: Kaggle Birds 525 Species
- **Więcej augmentacji**: Mixup, CutMix, RandomErasing
- **Większy model**: EfficientNet-B3/B4
- **TTA**: Test Time Augmentation
- **Interfejs**: Gradio lub Streamlit

### Przydatne linki:
- [PyTorch Transfer Learning Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)
- [EfficientNet Paper](https://arxiv.org/abs/1905.11946)
- [Kaggle Birds Dataset](https://www.kaggle.com/datasets/gpiosenka/100-bird-species)